# Import data and general packages

#### Install MeCab and dependencies, if necessary

#### Imports

In [66]:
import collections
import logging
import MeCab
import neologdn
import numpy as np
import os
import pandas as pd
import re
import time

from nltk.corpus import stopwords
from gensim.models.word2vec import Word2Vec

logging.basicConfig()
logging.root.setLevel(logging.INFO)

#### Get data

In [3]:
os.chdir("/home/andi/code/mochiyam/simply-japanese/data/2_RawData") # Change user name to run on your machine
os.getcwd()

'/home/andi/code/mochiyam/simply-japanese/data/2_RawData'

In [4]:
!ls

Combined_85K_10000.xlsx  SNOW_T15_10000.xlsx  SNOW_T23_10000.xlsx
Combined_85K_1000.xlsx	 SNOW_T15_1000.xlsx   SNOW_T23_1000.xlsx
Combined_85K_150.xlsx	 SNOW_T15_150.xlsx    SNOW_T23_150.xlsx
Combined_85K.xlsx	 SNOW_T15.xlsx	      SNOW_T23.xlsx


In [5]:
data = "SNOW_T15_10000.xlsx"
df = pd.read_excel(data)
data = "SNOW_T15_150.xlsx"
df150 = pd.read_excel(data)

In [6]:
df.head()

,#日本語(原文),#やさしい日本語,#英語(原文)
0,父は私が外国へ行くことを承知した。,父は私が外国へ行くことを許した。,my father consented to my going abroad .
1,卑屈な奴。,自分のことをダメだと考える人。,don 't be a dog .
2,それは本当のはずはない。,それは本当のはずはない。,it cannot be true .
3,車がそんなに混んでなければ問題ないでしょう。,車がそんなに混んでなければ問題ないでしょう。,we shouldn 't have any trouble if the traffic ...
4,２時間も待たされた。,２時間も待った。,i was kept waiting for as long as two hours .


In [7]:
X = df[df.keys()[0]]
y = df[df.keys()[1]]
X150 = df150[df150.keys()[0]]
y150 = df150[df150.keys()[1]]

In [8]:
X[0], y[0]

('父は私が外国へ行くことを承知した。', '父は私が外国へ行くことを許した。')

# Janome

In [9]:
from janome.tokenizer import Tokenizer

Simple Tokenizer

In [10]:
t = Tokenizer()
for token in t.tokenize(X[0]):
    print(token)

父	名詞,一般,*,*,*,*,父,チチ,チチ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
外国	名詞,一般,*,*,*,*,外国,ガイコク,ガイコク
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行く	動詞,自立,*,*,五段・カ行促音便,基本形,行く,イク,イク
こと	名詞,非自立,一般,*,*,*,こと,コト,コト
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
承知	名詞,サ変接続,*,*,*,*,承知,ショウチ,ショーチ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。


Even simpler tokenizer<br>
wakati (分ち書き）returns only surface tokens

In [11]:
for token in t.tokenize(X[0], wakati=True):
    print(token)

父
は
私
が
外国
へ
行く
こと
を
承知
し
た
。


# Identify words to be replaced / ignored

## Data

Our df are 10000 random segments from the SNOW T15 dataset.
df150 are 150 random segments from the SNOW T15 dataset.

In [245]:
df.head()

,#日本語(原文),#やさしい日本語,#英語(原文)
0,父は私が外国へ行くことを承知した。,父は私が外国へ行くことを許した。,my father consented to my going abroad .
1,卑屈な奴。,自分のことをダメだと考える人。,don 't be a dog .
2,それは本当のはずはない。,それは本当のはずはない。,it cannot be true .
3,車がそんなに混んでなければ問題ないでしょう。,車がそんなに混んでなければ問題ないでしょう。,we shouldn 't have any trouble if the traffic ...
4,２時間も待たされた。,２時間も待った。,i was kept waiting for as long as two hours .


X and y are the 0th and 1st columns of df,

In [ ]:
X[0], y[0], len(X), len(y)

In [14]:
node = tagger.parseToNode(text)
while node:
    features = node.feature.split(",")
    original = features[6]
    print('node.surface={},\n node.feature={} => original = {}, \n\n'.format(node.surface,node.feature,original))
    node = node.next

node.surface=,
 node.feature=BOS/EOS,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,* => original = *, 


node.surface=月曜,
 node.feature=名詞,普通名詞,副詞可能,*,*,*,ゲツヨウ,月曜,月曜,ゲツヨー,月曜,ゲツヨー,漢,*,*,*,*,ゲツヨウ,ゲツヨウ,ゲツヨウ,ゲツヨウ,*,*,"3,0",C2,* => original = ゲツヨウ, 


node.surface=日,
 node.feature=名詞,普通名詞,副詞可能,*,*,*,ヒ,日,日,ヒ,日,ヒ,和,ヒ混合,基本形,*,*,ヒ,ヒ,ヒ,ヒ,*,*,"0,1",C3,* => original = ヒ, 


node.surface=まで,
 node.feature=助詞,副助詞,*,*,*,*,マデ,まで,まで,マデ,まで,マデ,和,*,*,*,*,マデ,マデ,マデ,マデ,*,*,*,"名詞%F2@1,形容詞%F2@1,動詞%F2@1",* => original = マデ, 


node.surface=に,
 node.feature=助詞,格助詞,*,*,*,*,ニ,に,に,ニ,に,ニ,和,*,*,*,*,ニ,ニ,ニ,ニ,*,*,*,名詞%F1,* => original = ニ, 


node.surface=この,
 node.feature=連体詞,*,*,*,*,*,コノ,此の,この,コノ,この,コノ,和,*,*,*,*,コノ,コノ,コノ,コノ,*,*,0,*,* => original = コノ, 


node.surface=仕事,
 node.feature=名詞,普通名詞,サ変可能,*,*,*,シゴト,仕事,仕事,シゴト,仕事,シゴト,和,*,*,*,*,シゴト,シゴト,シゴト,シゴト,*,*,0,C2,* => original = シゴト, 


node.surface=を,
 node.feature=助詞,格助詞,*,*,*,*,ヲ,を,を,オ,を,オ,和,*,*,*,*,ヲ,ヲ,ヲ,ヲ,*,*,*,"動詞%F2@0,名詞%F1,形容詞%F2@-1",* => original = ヲ, 


node.su

## Load word2Vec model

In [12]:
model_path = '/home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model'
model = Word2Vec.load(model_path)

INFO:gensim.utils:loading Word2Vec object from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model
INFO:gensim.utils:loading wv recursively from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model.wv.* with mmap=None
INFO:gensim.utils:loading syn0 from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model.wv.syn0.npy with mmap=None
INFO:gensim.utils:loading syn1neg from /home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model.syn1neg.npy with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': '/home/andi/code/PlayGround/ShiroYagi/word2vec.gensim.model', 'datetime': '2023-02-18T15:58:43.854889', 'gensim': '4.3.0', 'python': '3.8.12 (default, Oct  2 2022, 16:53:09) \n[GCC 11.2.0]', 'platform': 'Linux-5.15.79.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'loaded'}


In [13]:
wv = model.wv

## Generate simp_term list

In [14]:
def clean_data(data):
    """
    Remove English column, rename columns
    """
    data = df.drop(columns=['#英語(原文)'])
    data.rename(columns={"#日本語(原文)": "original", "#やさしい日本語": "simplified"}, inplace=True)
    
    return data

df_clean = clean_data(df)

In [15]:
# Counts all the independent word 自立語
_stopwords = stopwords.words('japanese.txt')

def count_all_words(docs, col='original'):
    all_words = collections.Counter()
    t = MeCab.Tagger("-O wakati")
    for idx, row in docs.iterrows():
        text = row[col]
        node = t.parseToNode(text).next
        while node.next:
            part_of_speech = node.feature.split(',')[0]   
            if part_of_speech in ["助動詞", "助詞", "補助記号"] or node.surface in _stopwords:
                node = node.next
                continue
            all_words[node.surface] += 1
            node = node.next
            
    return all_words

In [16]:
def create_terms(data):
    # corpus of all sentences, where simplication differs from original
    diff_corpus_df = data[data['original'] != data['simplified']]
    # Count all words in the original and simplified column. Ignore stopwords and certain POS
    original_temp_list = count_all_words(diff_corpus_df, 'original')
    diff_temp = count_all_words(diff_corpus_df, 'simplified')
    diff_temp.subtract(original_temp_list)
    diff_temp_df = pd.DataFrame(dict(diff_temp).items(), columns=['word', 'count'])
    diff_temp_df = diff_temp_df[diff_temp_df['count'] >= 0]
    # Make this a list
    term_list = diff_temp_df['word'].to_list()
    
    return term_list

term_list = create_terms(df_clean)
print(len(term_list))

1780


In [17]:
len(term_list)

1780

## Iterate over data and remove terms

In [102]:
# old version that could not handle romaji

def replace_terms(data, term_list=term_list, wv=wv):
    """
    1. Identify every POS in a sentence and if it should be replaced
    2. Use the pre-trained Word2Vec model to get a term from term_list with closest distance to POS
    3. Replace POS in sentence
    4. Add new sentence to dataframe in column "prediction"
    
    input:
    data, np.series
    term_list, list of simplified terms
    wv, word2vec model.wv
    
    output:
    prediction, np.series
    """
    logging.root.setLevel(logging.DEBUG)
    
    start = time.time()
    # Make sure the data is a series, not a df or list
    try:
        assert type(data) == pd.core.series.Series
        logging.info("Data file type OK")
    except:
        print("Data file type is NOT a pd.series")
    
    pos_list = ("名詞", "動詞", "代名詞") # POS (part of speech) that will possibly be removed
    threshold = 0.5 # Threshold of similarity, over which a term will be replaced
    t = MeCab.Tagger()
    numbers_dict = {
        "0":"零",
        "1":"一",
        "2":"二",
        "3":"三",
        "4":"四",
        "5":"五",
        "6":"六",
        "7":"七",
        "8":"八",
        "9":"九"
    }
    counter = collections.Counter()
    prediction = data.copy()
    assert len(prediction) == len(data)     # Make sure prediction and data have the same size
    
    # Iterate over every sentence in the dataset
    for idx, row in data.items():
        row = neologdn.normalize(row)
        logging.debug(f"Currrent sentence: {row}")
        numerals = sum(c.isdigit() for c in row)
        if numerals > 0:
            for i in range(numerals):
                for entry in numbers_dict:
                    row = row.replace(entry, numbers_dict[entry])
        sentence = []
        # Iterate over every word in the sentence
        node = t.parseToNode(row).next
        while node.next:
            word = node.feature.split(',')[8]
            part_of_speech = node.feature.split(',')[0]
            # If POS is not noun, pronoun or verb: add word to list and continue
            if part_of_speech not in pos_list:
                sentence.append(word)
            else:
                # If the term is already in the term list: do not replace, add word to list and continue
                if word in term_list:
                    sentence.append(word)
                else:
                    # Replace word with closest word from term list
                    try:
                        if wv.most_similar(word)[0][1] > threshold:
                            closest_word = wv.most_similar(word)[0][0]
                            sentence.append(closest_word)
                        else:
                            sentence.append(word)
                    except KeyError as e:
                        sentence.append(word)
                        logging.warning(f"{e}. Term will not be replaced.")
            counter[node.surface] += 1
            node = node.next
        logging.debug(sentence)
        prediction[idx] = "".join(sentence)
    
    assert len(data) == len(prediction)  # Make sure prediction and data have the same size
    end = time.time()
    logging.info(end-start)
    return prediction

In [103]:
# new version that can handle romaji, also changed the way it handles numerals

def is_romaji(string):
    """
    returns True if string contains Romaji or a number, False if not.
    """
    alphanum_full = r'[A-z0-9]'
    if re.findall(alphanum_full, string):
        return True
    return False

def replace_terms(data, term_list=term_list, wv=wv):
    """
    1. Identify every POS in a sentence and if it should be replaced
    2. Use the pre-trained Word2Vec model to get a term from term_list with closest distance to POS
    3. Replace POS in sentence
    4. Add new sentence to dataframe in column "prediction"
    
    input:
    data, np.series
    term_list, list of simplified terms
    wv, word2vec model.wv
    
    output:
    prediction, np.series
    """
    logging.root.setLevel(logging.DEBUG)
    
    start = time.time()
    # Make sure the data is a series, not a df or list
    try:
        assert type(data) == pd.core.series.Series
        logging.info("Data file type OK")
    except:
        print("Data file type is NOT a pd.series")
    
    pos_list = ("名詞", "動詞", "代名詞") # POS (part of speech) that will possibly be removed
    threshold = 0.5 # Threshold of similarity, over which a term will be replaced
    t = MeCab.Tagger()
    counter = collections.Counter()
    prediction = data.copy()
    assert len(prediction) == len(data)     # Make sure prediction and data have the same size
    
    # Iterate over every sentence in the dataset
    for idx, row in data.items():
        row = neologdn.normalize(row)
        logging.debug(f"Currrent sentence: {row}")
        sentence = []
        # Iterate over every word in the sentence
        node = t.parseToNode(row).next
        while node.next:
            if is_romaji(node.surface):
                sentence.append(node.surface)
                node = node.next
            word = node.feature.split(',')[8]
            part_of_speech = node.feature.split(',')[0]
            # If POS is not noun, pronoun or verb: add word to list and continue
            if part_of_speech not in pos_list:
                sentence.append(word)
            else:
                # If the term is already in the term list: do not replace, add word to list and continue
                if word in term_list:
                    sentence.append(word)
                else:
                    # Replace word with closest word from term list
                    try:
                        if wv.most_similar(word)[0][1] > threshold:
                            closest_word = wv.most_similar(word)[0][0]
                            sentence.append(closest_word)
                        else:
                            sentence.append(word)
                    except KeyError as e:
                        sentence.append(word)
                        logging.warning(f"{e}. Term will not be replaced.")
            counter[node.surface] += 1
            node = node.next
        logging.debug(sentence)
        prediction[idx] = "".join(sentence)
    
    assert len(data) == len(prediction)  # Make sure prediction and data have the same size
    end = time.time()
    logging.info(end-start)
    return prediction

In [101]:
predictions = replace_terms(X)

INFO:root:Data file type OK
DEBUG:root:Currrent sentence: 父は私が外国へ行くことを承知した。
DEBUG:root:['父', 'は', '貴方', 'が', '外国', 'へ', '行く', '事', 'を', '弁解', '図っ', 'た', '。']
DEBUG:root:Currrent sentence: 卑屈な奴。
DEBUG:root:['尊大', 'な', '貴様', '。']
DEBUG:root:Currrent sentence: それは本当のはずはない。
DEBUG:root:['これ', 'は', '本当', 'の', 'はず', 'は', 'ない', '。']
DEBUG:root:Currrent sentence: 車がそんなに混んでなければ問題ないでしょう。
DEBUG:root:['車', 'が', 'そんな', 'に', '混ん', 'で', 'なけれ', 'ば', '問題', 'ない', 'でしょう', '。']
DEBUG:root:Currrent sentence: 2時間も待たされた。
DEBUG:root:['2', '時間', 'も', 'やらさ', 'れ', 'た', '。']
DEBUG:root:Currrent sentence: どうぞ午後に私の家へ来てください。
DEBUG:root:['どうぞ', '午後', 'に', '貴方', 'の', '家', 'へ', '来', 'て', 'ください', '。']
DEBUG:root:Currrent sentence: 彼はいつも父親の陰口を言います。
DEBUG:root:['彼自身', 'は', 'いつ', 'も', '母親', 'の', '罵っ', 'を', '言い', 'ます', '。']
DEBUG:root:Currrent sentence: 彼女はもう少しで遅刻するところだった。
DEBUG:root:['彼女自身', 'は', 'もう', '少し', 'で', '早退', '促す', '事', 'だっ', 'た', '。']
DEBUG:root:Currrent sentence: 電話を切らずにそのまま待ってください。
DEBUG:root:['電話', 'を', '切ら', 

DEBUG:root:Currrent sentence: 君の心を傷つけるつもりはなかった。
DEBUG:root:['あたし', 'の', '心', 'を', '犯す', 'つもり', 'は', 'なかっ', 'た', '。']
DEBUG:root:Currrent sentence: これはよい教科書だ。
DEBUG:root:['それ', 'は', 'よい', '教科', '書', 'だ', '。']
DEBUG:root:Currrent sentence: 私たちはその機会を最大限に利用した。
DEBUG:root:['貴方', 'たち', 'は', 'その', '機会', 'を', '最大規模', '限', 'に', '利用', '図っ', 'た', '。']
DEBUG:root:Currrent sentence: 「風と共に去りぬ」を読む。
DEBUG:root:['「', '風', 'と', '共', 'に', '去っ', 'ぬ', '」', 'を', '読む', '。']
DEBUG:root:Currrent sentence: 湖は直径3マイルある。
DEBUG:root:['峡谷', 'は', '長さ', '3', '㎞', 'あり', '。']
DEBUG:root:Currrent sentence: 彼女は知らせを聞いて驚いた。
DEBUG:root:['彼女自身', 'は', '知ら', 'せ', 'を', '聞い', 'て', '驚い', 'た', '。']
DEBUG:root:Currrent sentence: 僕は彼女の歌のピアノ伴奏をした。
DEBUG:root:['俺', 'は', '彼女自身', 'の', '歌', 'の', 'ピアノ', '合奏', 'を', '図っ', 'た', '。']
DEBUG:root:Currrent sentence: あなたにお会いできて嬉しい。
DEBUG:root:['あなた', 'に', 'お', '会い', '出来', 'て', '嬉しい', '。']
DEBUG:root:Currrent sentence: 名案を思いついたぞ。
DEBUG:root:['シュトヘル', 'を', '思い付い', 'た', 'ぞ', '。']
DEBUG:root:Currrent 

DEBUG:root:Currrent sentence: 君は学校で何の科目を取っていますか。
DEBUG:root:['あたし', 'は', '学校', 'で', '幾', 'の', '教科', 'を', '取っ', 'て', 'き', 'ます', 'か', '。']
DEBUG:root:Currrent sentence: きっと私はこの宿題をします。
DEBUG:root:['きっと', '貴方', 'は', 'この', '宿題', 'を', '図っ', 'ます', '。']
DEBUG:root:Currrent sentence: 彼はすぐ怒るので嫌いだ。
DEBUG:root:['彼自身', 'は', 'すぐ', '怒る', 'の', 'で', '嫌い', 'だ', '。']
DEBUG:root:Currrent sentence: 彼はこの家の主人です。
DEBUG:root:['彼自身', 'は', 'この', '家', 'の', '女中', 'です', '。']
DEBUG:root:Currrent sentence: 例えば、君は英語が好きですか。
DEBUG:root:['例えば', '、', 'あたし', 'は', '英語', 'が', '好き', 'です', 'か', '。']
DEBUG:root:Currrent sentence: 彼の意見は我々の意見とはまったく違っている。
DEBUG:root:['彼自身', 'の', '意見', 'は', 'われわれ', 'の', '意見', 'と', 'は', 'まったく', '異なっ', 'て', 'おり', '。']
DEBUG:root:Currrent sentence: 彼女は通りを横切った。
DEBUG:root:['彼女自身', 'は', '通り', 'を', 'かすめ', 'た', '。']
DEBUG:root:Currrent sentence: いいカメラを買った。
DEBUG:root:['いい', 'カメラ', 'を', '買っ', 'た', '。']
DEBUG:root:Currrent sentence: すべてのバスは満員です。
DEBUG:root:['全て', 'の', 'バス', 'は', '立ち見', 'です', '。']
DEBUG:root

DEBUG:root:['今日', 'は', '幾', 'か', '変わっ', 'た', '事', 'が', 'だっ', 'た', '?']
DEBUG:root:Currrent sentence: その船は私たちの船に迫ってきた。
DEBUG:root:['その', '船', 'は', '貴方', 'たち', 'の', '船', 'に', '近づい', 'て', 'い', 'た', '。']
DEBUG:root:Currrent sentence: あたり1面花だった。
DEBUG:root:['あたり', '1', '層分', '花', 'だっ', 'た', '。']
DEBUG:root:Currrent sentence: 靴を脱ぎなさい。
DEBUG:root:['靴', 'を', '脱ぐ', '下さい', '。']
DEBUG:root:Currrent sentence: 姉が僕に新しい時計を見せてくれた。
DEBUG:root:['姉', 'が', '俺', 'に', '新しい', '時計', 'を', '見せ', 'て', 'くれ', 'た', '。']
DEBUG:root:Currrent sentence: 田舎は春がきれいだ。
DEBUG:root:['都会', 'は', '春', 'が', 'きれい', 'だ', '。']
DEBUG:root:Currrent sentence: 明日君は彼女に会えるでしょう。
DEBUG:root:['明日', '君', 'は', '彼女自身', 'に', 'ついて行く', 'でしょう', '。']
DEBUG:root:Currrent sentence: あなたは何度も同じ誤りばかりしている。
DEBUG:root:['あなた', 'は', '幾', '度', 'も', '同じ', '事実', 'ばかり', '図っ', 'て', 'おり', '。']
DEBUG:root:Currrent sentence: 私達は良く一緒にお昼を食べます。
DEBUG:root:['貴方', '達', 'は', '良く', '一緒', 'に', 'お', '昼', 'を', '食べ', 'ます', '。']
DEBUG:root:Currrent sentence: 彼女はうそをつくことはしない。
DEBU

DEBUG:root:['彼自身', 'が', '来る', '事', 'を', '見つけ出す', '為', 'に', '電話', 'を', '図っ', 'た', '。']
DEBUG:root:Currrent sentence: 彼の家は丸太でできていた。
DEBUG:root:['彼自身', 'の', '家', 'は', '杭', 'で', '出来', 'て', 'き', 'た', '。']
DEBUG:root:Currrent sentence: その単語を辞書で調べてごらん。
DEBUG:root:['その', '語句', 'を', '辞書', 'で', '調べ', 'て', '下さっ', '。']
DEBUG:root:Currrent sentence: 私はやるべきことを山積みにしている。
DEBUG:root:['貴方', 'は', 'やる', 'べき', '事', 'を', '箱詰め', 'に', '図っ', 'て', 'おり', '。']
DEBUG:root:Currrent sentence: 私の友人はだれもコーヒーを飲まない。
DEBUG:root:['貴方', 'の', '旧友', 'は', 'いま', 'も', 'コーヒー', 'を', '飲ま', 'ない', '。']
DEBUG:root:Currrent sentence: その仕事を彼にゆずった。
DEBUG:root:['その', '仕事', 'を', '彼自身', 'に', '放り出す', 'た', '。']
DEBUG:root:Currrent sentence: 彼は来週会議にくるでしょうか。
DEBUG:root:['彼自身', 'は', '来週', '会議', 'に', 'くる', 'でしょう', 'か', '。']
DEBUG:root:Currrent sentence: 休みになったらいつか御出かけになって下さい。
DEBUG:root:['休み', 'に', 'なり', 'たら', 'いつ', 'か', '御', '出掛け', 'に', 'なり', 'て', 'ください', '。']
DEBUG:root:Currrent sentence: 我々は次の日曜日に出発することに決めた。
DEBUG:root:['われわれ', 'は', '次', 'の', 

...

DEBUG:root:['その', '試験', 'の', '結果', 'は', 'われわれ', 'の', '期待', 'に', 'もうい', 'なかっ', 'た', '。']
DEBUG:root:Currrent sentence: この問題を解くのはほとんど不可能である。
DEBUG:root:['この', '問題', 'を', '導き出す', 'の', 'は', '殆ど', '不', '可能', 'で', 'あり', '。']
DEBUG:root:Currrent sentence: 彼は金持ちだが一生懸命働いている。
DEBUG:root:['彼自身', 'は', '大金持ち', 'だ', 'が', '人生', '懸命', '働い', 'て', 'おり', '。']
DEBUG:root:Currrent sentence: 毎朝犬の運動に公園を散歩する。
DEBUG:root:['昼食', '犬', 'の', '運動', 'に', '公園', 'を', '散策', '促す', '。']
DEBUG:root:Currrent sentence: 私はトムに電話した。
DEBUG:root:['貴方', 'は', 'トム', 'に', '電話', '図っ', 'た', '。']
DEBUG:root:Currrent sentence: ちゃんと朝ご飯を食べて行きなさい。
DEBUG:root:['ちゃんと', '朝ご飯', 'を', '食べ', 'て', '行き', '下さい', '。']
DEBUG:root:Currrent sentence: 今日は昨日よりもっとずっとたくさんの学生が教室にいる。
DEBUG:root:['今日', 'は', '昨日', 'より', 'もっと', 'ずっと', 'たくさん', 'の', '学生', 'が', '教室', 'に', 'おり', '。']
DEBUG:root:Currrent sentence: 彼はその火事で焼け死んだ。
DEBUG:root:['彼自身', 'は', 'その', '火事', 'で', '泣き叫ん', 'だ', '。']
DEBUG:root:Currrent sentence: 車に鍵をかけておかなかったの?
DEBUG:root:['車', 'に', '鍵', 'を', 'かけ'

DEBUG:root:Currrent sentence: そのような方法は時代遅れである。
DEBUG:root:['その', 'よう', 'な', '方法', 'は', '末期', '遅れ', 'で', 'あり', '。']
DEBUG:root:Currrent sentence: 7時より前にここに来るように。
DEBUG:root:['7', '時', 'より', '前', 'に', '今日', 'に', '来る', 'よう', 'に', '。']
DEBUG:root:Currrent sentence: 君はどこにいたの?
DEBUG:root:['あたし', 'は', '何処', 'に', 'き', 'た', 'の', '?']
DEBUG:root:Currrent sentence: 彼女はとても美しい。その上、とても賢い。
DEBUG:root:['彼女自身', 'は', 'とても', '美しい', '。', 'その', '上', '、', 'とても', '賢い', '。']
DEBUG:root:Currrent sentence: どんなに年をとっても学ぶことはできる。
DEBUG:root:['どんな', 'に', '年', 'を', 'とっ', 'て', 'も', '学ぶ', '事', 'は', '出来る', '。']
DEBUG:root:Currrent sentence: その知らせで彼女は突然泣き出した。
DEBUG:root:['その', '報せ', 'で', '彼女自身', 'は', '突然', '泣き', '出し', 'た', '。']
DEBUG:root:Currrent sentence: 私の父と兄はこの工場で働いている。
DEBUG:root:['貴方', 'の', '父', 'と', '兄', 'は', 'この', '製鉄所', 'で', '働い', 'て', 'おり', '。']
DEBUG:root:Currrent sentence: 太陽は地球からとても遠い。
DEBUG:root:['太陽', 'は', '地球', 'から', 'とても', '遠い', '。']
DEBUG:root:Currrent sentence: 先生にこの事は知らせましたか。
DEBUG:root:['先生', 'に', '

DEBUG:root:Currrent sentence: 何か事故があったらしい。
DEBUG:root:['幾', 'か', '事故', 'が', 'だっ', 'た', 'らしい', '。']
DEBUG:root:Currrent sentence: 彼女の母は夕食を作るのに忙しかった。
DEBUG:root:['彼女自身', 'の', '母', 'は', '夕食', 'を', '作る', 'の', 'に', '忙しかっ', 'た', '。']
DEBUG:root:Currrent sentence: スミス家の人達はみんないい人です。
DEBUG:root:['スミス', '家', 'の', '人', '達', 'は', 'みんな', 'いい', '人', 'です', '。']
DEBUG:root:Currrent sentence: ご遠慮なくケーキをお取りください。
DEBUG:root:['ご', '立腹', 'なく', 'ケーキ', 'を', 'お', '取り', 'ください', '。']
DEBUG:root:Currrent sentence: 問題はいつ始めるかだ。
DEBUG:root:['問題', 'は', 'いつ', '始める', 'か', 'だ', '。']
DEBUG:root:Currrent sentence: ある日彼はその道路を歩いていました。
DEBUG:root:['ある', '日', '彼自身', 'は', 'その', '道路', 'を', '歩い', 'て', 'き', 'まし', 'た', '。']
DEBUG:root:Currrent sentence: その老人は私に時間を尋ねた。
DEBUG:root:['その', '浮浪者', 'は', '貴方', 'に', '時間', 'を', '尋ね', 'た', '。']
DEBUG:root:Currrent sentence: 眠っているの、トム。
DEBUG:root:['横たわっ', 'て', 'おり', 'の', '、', 'トム', '。']
DEBUG:root:Currrent sentence: 犬に近寄るな。
DEBUG:root:['犬', 'に', 'やり過ごす', 'な', '。']
DEBUG:root:Currrent sentenc

DEBUG:root:Currrent sentence: 他人を軽蔑するな。
DEBUG:root:['他者', 'を', '嫌悪', '促す', 'な', '。']
DEBUG:root:Currrent sentence: 女の子達はみんなお互いに助け合った。
DEBUG:root:['男の子', '達', 'は', 'みんな', 'お', '互い', 'に', '触れ合っ', 'た', '。']
DEBUG:root:Currrent sentence: 私はもうほとんど歩けない。
DEBUG:root:['貴方', 'は', 'もう', '殆ど', '行け', 'ない', '。']
DEBUG:root:Currrent sentence: 今日は休みです。
DEBUG:root:['今日', 'は', '休み', 'です', '。']
DEBUG:root:Currrent sentence: 彼女は金持ちで、更によいことには、大変美しい。
DEBUG:root:['彼女自身', 'は', '大金持ち', 'で', '、', '更に', 'よい', '事', 'に', 'は', '、', '大変', '美しい', '。']
DEBUG:root:Currrent sentence: 遊びましょう。
DEBUG:root:['楽しみ', 'ましょう', '。']
DEBUG:root:Currrent sentence: 私は金を持ち合わせていない。
DEBUG:root:['貴方', 'は', '金', 'を', '併せ持っ', 'て', 'き', 'ない', '。']
DEBUG:root:Currrent sentence: 天気がよければ私達は明朝出発します。
DEBUG:root:['天気', 'が', 'よけれ', 'ば', '貴方', '達', 'は', '元朝', '出発', '図っ', 'ます', '。']
DEBUG:root:Currrent sentence: 今は仕事中なのであとから電話します。
DEBUG:root:['今', 'は', '仕事', '中', 'な', 'の', 'で', 'あと', 'から', '電話', '図っ', 'ます', '。']
DEBUG:root:Currrent sentence: 君はどこへで

DEBUG:root:['彼自身', 'の', '会社', 'は', '彼自身', 'が', '統治', '図っ', 'て', 'おり', '。']
DEBUG:root:Currrent sentence: 昨日台風が来ました。
DEBUG:root:['昨日', '豪雨', 'が', '来', 'まし', 'た', '。']
DEBUG:root:Currrent sentence: 彼が帰国の腹を決めたのは病気だったからだ。
DEBUG:root:['彼自身', 'が', '一時帰国', 'の', '腹', 'を', '決め', 'た', 'の', 'は', '病気', 'だっ', 'た', 'から', 'だ', '。']
DEBUG:root:Currrent sentence: 私は父に反対してまで彼女の味方になった。
DEBUG:root:['貴方', 'は', '父', 'に', '反対', '図っ', 'て', 'まで', '彼女自身', 'の', '味方', 'に', 'なり', 'た', '。']
DEBUG:root:Currrent sentence: できればそうしたいけど。
DEBUG:root:['できれ', 'ば', 'そう', '図っ', 'たい', 'けど', '。']
DEBUG:root:Currrent sentence: 私は寒さに弱い。
DEBUG:root:['貴方', 'は', '寒', 'さ', 'に', '弱い', '。']
DEBUG:root:Currrent sentence: 明日雨ならば行きません。
DEBUG:root:['明日', '雨', 'なら', 'ば', '行き', 'ませ', 'ん', '。']
DEBUG:root:Currrent sentence: 私たちはかなりいろんな所を訪ねました。
DEBUG:root:['貴方', 'たち', 'は', 'かなり', 'いろんな', '所', 'を', '訪ね', 'まし', 'た', '。']
DEBUG:root:Currrent sentence: 彼は長いこと私を待たせておいた。
DEBUG:root:['彼自身', 'は', '長い', '事', '貴方', 'を', '待た', 'せ', 'て', 'おい', 'た', '。']


In [83]:
text ="今112日からcd丸日EFお願いします。"
t = MeCab.Tagger()

In [85]:
all_terms = collections.Counter()
node = t.parseToNode(text).next
while node.next:
    part_of_speech = node.feature
    print(node.surface.isalpha(), node.surface)
    all_terms[node.surface] += 1
    node = node.next

True 今
False 112
True 日
True から
True cd
True 丸
True 日
True EF
True お
True 願い
True し
True ます
False 。


In [108]:
predictions.str.find("cd").sort_values(ascending=False)
predictions[847]

'彼女自身はcdを買ったと貴方に言った。'

In [96]:


def is_romaji(string):
    """
    returns True if string is Romaji, False if not.
    """
    alphanum_full = r'[A-z0-9]'
    if re.findall(alphanum_full, string):
        return True
    return False

text1 = "らりるれろ"
text2 = "54046"
text3 = "ahsg"

print(text1, is_romaji(text1))
print(text2, is_romaji(text2))
print(text3, is_romaji(text3))

らりるれろ False
54046 True
ahsg True


In [109]:
wv.most_similar("し")

[('図っ', 0.6891764998435974),
 ('行なっ', 0.6623058319091797),
 ('促し', 0.6435315012931824),
 ('行っ', 0.641490638256073),
 ('おこなっ', 0.6279498338699341),
 ('はかっ', 0.6224112510681152),
 ('受け', 0.613266110420227),
 ('努め', 0.6071730256080627),
 ('目指し', 0.6017763614654541),
 ('見送っ', 0.5945292115211487)]